In [1]:
import ee

# To run the code, it is necessary to register and authenticate in Google Earth Engine!
ee.Authenticate()
ee.Initialize(project='time-series-462612')



## Helper Functions

In [2]:
# --------------------------- S2 PREPROCESS ---------------------------

def preproS2(ic, cloud_prob):
    """
    Args:
      ic: ee.ImageCollection of Sentinel-2
      cloud_prob: numeric threshold for Cloud probability
    Returns:
      ee.ImageCollection with optical bands (B.*) masked by clouds & land cover.
    """
    def _map(img):
        img = ee.Image(img)
        cloud_img = ee.Image(img.get('Cloud'))
        mask_cloud = cloud_img.select('probability').lte(cloud_prob)
        mask_land = img.select('SCL').gt(1).And(img.select('SCL').lt(9))
        mask = mask_cloud.And(mask_land)
        return (img.select('B.*')
                   .updateMask(mask)
                   .copyProperties(img, ['system:time_start']))
    return ee.ImageCollection(ic.map(_map))

# --------------------------- S2 INDICES ---------------------------

def _ndvi(image):
    ndvi_val = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi_val)

def _srnirr(image):
    srnirr_val = image.expression(
        '(B8/B4)', {'B4': image.select('B4'), 'B8': image.select('B8')}
    ).rename('SRNIRR')
    return image.addBands(srnirr_val)

def _direswir(image):
    direswir_val = image.expression(
        '(B4 - B11)',
        {'B11': image.select('B11').multiply(0.0001),
         'B4':  image.select('B4').multiply(0.0001)}
    ).rename('DIRESWIR')
    return image.addBands(direswir_val)

def _lcci(image):
    lcci_val = image.expression(
        '(B7/B5)', {'B7': image.select('B7'), 'B5': image.select('B5')}
    ).rename('LCCI')
    return image.addBands(lcci_val)

def _gnvi(image):
    gndvi_val = image.normalizedDifference(['B8', 'B3']).rename('gndvi')
    return image.addBands(gndvi_val)

def _evi(image):
    evi_val = image.expression(
        '2.4 * ((B8 - B4) / (B8 + B4 + 1))',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001)}
    ).rename('EVI2')
    return image.addBands(evi_val)

def _lai(image):
    # LAI = 3.618 * EVI - 0.118, with EVI = 2.5 * (B8 - B4) / (B8 + 6B4 - 7.5B2 + 1)
    lai_val = image.expression(
        '3.618 * (2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))) - 0.118',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001),
         'B2': image.select('B2').multiply(0.0001)}
    ).rename('LAI')
    return image.addBands(lai_val)

def _laig(image):
    # LAIg = 5.405 * ((B8A - B5)/(B8A + B5)) - 0.114
    laig_val = image.expression(
        '5.405 * ((B8A - B5) / (B8A + B5)) - 0.114',
        {'B8A': image.select('B8A').multiply(0.0001),
         'B5':  image.select('B5').multiply(0.0001)}
    ).rename('LAIg')
    return image.addBands(laig_val)

def _msavi(image):
    msavi_val = image.expression(
        '0.5 * (2 * B8 + 1 - sqrt((2 * B8 + 1) ** 2 - 8 * (B8 - B4)))',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001)}
    ).rename('MSAVI')
    return image.addBands(msavi_val)

def _savi(image):
    savi_val = image.expression(
        '(1.5 * ((B8 - B4) / (B8 + B4 + 0.5)))',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001)}
    ).rename('SAVI')
    return image.addBands(savi_val)

def _bsi(image):
    bsi_val = image.expression(
        '((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2))',
        {'B11': image.select('B11'),
         'B4':  image.select('B4'),
         'B8':  image.select('B8'),
         'B2':  image.select('B2')}
    ).rename('BSI')
    return image.addBands(bsi_val)

def _ndti(image):
    ndti_val = image.normalizedDifference(['B11', 'B12']).rename('NDTI')
    return image.addBands(ndti_val)

def _ndbi(image):
    ndbi_val = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    return image.addBands(ndbi_val)

def _blfei(image):
    blfei_val = image.expression(
        '((B3 + B4 + B12) / 3 - B11) / (((B3 + B4 + B12) / 3) + B11)',
        {'B3':  image.select('B3'),
         'B4':  image.select('B4'),
         'B12': image.select('B12'),
         'B11': image.select('B11')}
    ).rename('BLFEI')
    return image.addBands(blfei_val)

def _mndwi(image):
    mndwi_val = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    return image.addBands(mndwi_val)

def _ndwi(image):
    ndwi_val = image.normalizedDifference(['B8', 'B11']).rename('NDWI')
    return image.addBands(ndwi_val)

def vegetation_indices(ic):
    """
    Apply all 16 indices and return the ImageCollection with added bands.
    """
    for f in (_ndvi, _srnirr, _direswir, _lcci, _gnvi, _evi, _lai, _laig,
              _msavi, _savi, _bsi, _ndti, _ndbi, _blfei, _mndwi, _ndwi):
        ic = ic.map(f)
    return ic

# --------------------------- S2 MONTHLY AGG ---------------------------

def generateS2month(months, data, yr):
    def _per_month(m):
        m = ee.Number(m)
        img = ee.Image(ee.Algorithms.If(
            m.lte(3),
            data.filter(ee.Filter.calendarRange(1, 3, 'month')).median()
                .set({'month': m, 'year': yr}),
            data.filter(ee.Filter.calendarRange(m, m, 'month')).median()
                .set({'month': m, 'year': yr})
        ))
        prefix = ee.String(str(yr) + '_')
        newnames = img.bandNames().map(lambda b: prefix.cat(ee.String(b)))
        return img.rename(newnames)
    return ee.ImageCollection.fromImages(months.map(_per_month))

def newNames(data, yr):
    return data.bandNames().map(lambda bd: ee.String(str(yr) + '_').cat(ee.String(bd)))

# --------------------------- S1 PREPROCESS & FEATURES ---------------------------

def preproS1(collection, geo):
    def _map(img):
        sigma0 = ee.Image(10).pow(ee.Image(img).divide(10.0))
        speckle = sigma0.focal_median(30, 'circle', 'meters')
        img_clip = speckle.clip(geo)
        return img_clip.copyProperties(img, ['system:time_start', 'system:time_end'])
    return collection.map(_map)

def s1_features(collection):
    """
    Add RVI, VV/VH, VV-VH, DPSVIm, NDPI bands.
    Assumes input images already have bands 'VV' and 'VH'
    """
    def _feat(image):
        vv = image.select('VV')
        vh = image.select('VH')
        rvi = image.expression('4 * VH / (VV + VH)', {'VV': vv, 'VH': vh}).rename('RVI')
        ratio = vv.divide(vh).rename('VV/VH')
        diff  = vv.subtract(vh).rename('VV-VH')
        dpsv  = image.expression('(VV*VV + VV*VH)/1.414213562', {'VV': vv, 'VH': vh}).rename('DPSVIm')
        ndpi  = image.expression('(VV - VH) / (VV + VH)', {'VV': vv, 'VH': vh}).rename('NDPI')
        return image.addBands([rvi, ratio, diff, dpsv, ndpi])
    return collection.map(_feat)

def generateS1month(collection, start, count, interval, units):
    """
    Slice collection into consecutive windows and take median per window.
    """
    origin = ee.Date(start)
    idx = ee.List.sequence(0, ee.Number(count).subtract(1))
    def _window(i):
        i = ee.Number(i)
        t0 = origin.advance(i.multiply(interval), units)
        t1 = origin.advance(i.add(1).multiply(interval), units)
        return (collection.filterDate(t0, t1).median()
                .set({'system:time_start': t0.millis(),
                      'system:time_end':   t1.millis()}))
    return ee.ImageCollection(idx.map(_window))

# --------------------------- GENERIC HELPERS ---------------------------

def clipic(collection, geo):
    return collection.map(lambda image: ee.Image(image).clip(geo.bounds()))

def rename_classes(img):
    img = ee.Image(img)
    mapping = [
        (0,100),(1,211),(2,212),(3,213),(4,214),(5,215),(6,216),(7,217),(8,218),(9,219),
        (10,221),(11,222),(12,223),(13,230),(14,231),(15,232),(16,233),(17,240),
        (18,250),(19,290),(20,300),(21,500),(22,600),(23,700),(24,800)
    ]
    for old, new in mapping:
        img = img.where(img.eq(old), new)
    return img

def postMap(img):
    img = ee.Image(img)
    patchsize = img.connectedPixelCount(30, eightConnected=False)
    filtered = img.focal_mode(radius=10, kernelType='square', units='pixels')
    return img.where(patchsize.lt(20), filtered)

# --------------------------- SLD & VIZ / LEGEND ---------------------------

sld_intervals = (
  '<RasterSymbolizer>'
  '<ColorMap type="intervals" extended="false">'
    '<ColorMapEntry color="#ff130f" quantity="100" label="Artificial"/>'
    '<ColorMapEntry color="#a57000" quantity="211" label="Common wheat"/>'
    '<ColorMapEntry color="#896054" quantity="212" label="Durum wheat"/>'
    '<ColorMapEntry color="#e2007c" quantity="213" label="Barley"/>'
    '<ColorMapEntry color="#aa007c" quantity="214" label="Rye"/>'
    '<ColorMapEntry color="#a05989" quantity="215" label="Oats"/>'
    '<ColorMapEntry color="#ffd300" quantity="216" label="Maize"/>'
    '<ColorMapEntry color="#00a8e2" quantity="217" label="Rice"/>'
    '<ColorMapEntry color="#d69ebc" quantity="218" label="Triticale"/>'
    '<ColorMapEntry color="#d69ebc" quantity="219" label="Other cereals"/>'
    '<ColorMapEntry color="#dda50a" quantity="221" label="Potatoes"/>'
    '<ColorMapEntry color="#a800e2" quantity="222" label="Sugar beet"/>'
    '<ColorMapEntry color="#00af49" quantity="223" label="Other roots crops"/>'
    '<ColorMapEntry color="#00af49" quantity="230" label="Other non permanent industrial crops"/>'
    '<ColorMapEntry color="#ffff00" quantity="231" label="Sunflower"/>'
    '<ColorMapEntry color="#d1ff00" quantity="232" label="Rape and turnip rape"/>'
    '<ColorMapEntry color="#267000" quantity="233" label="Soya"/>'
    '<ColorMapEntry color="#f2a377" quantity="240" label="Dry pulses, vegetables and flowers"/>'
    '<ColorMapEntry color="#e8bfff" quantity="250" label="Other fodder crops"/>'
    '<ColorMapEntry color="#000000" quantity="290" label="Bare arable land"/>'
    '<ColorMapEntry color="#93cc93" quantity="300" label="Woodland and shrubland type of vegetation"/>'
    '<ColorMapEntry color="#e8ffbf" quantity="500" label="Grassland"/>'
    '<ColorMapEntry color="#a89e7f" quantity="600" label="Bare land/lichens moss"/>'
    '<ColorMapEntry color="#0793de" quantity="700" label="Water"/>'
    '<ColorMapEntry color="#7cafaf" quantity="800" label="Wetlands"/>'
  '</ColorMap>'
  '</RasterSymbolizer>'
)

def Visualization(img):
    return ee.Image(img).sldStyle(sld_intervals)

def legend_items_from_sld():
    import re
    pattern = re.compile(r'<ColorMapEntry color="([^"]+)" quantity="[^"]+" label="([^"]+)"\/>')
    return [{'color': c, 'label': l} for (c, l) in pattern.findall(sld_intervals)]


## Train

In [3]:
# --------------------------------------------------
# 1. INPUT ASSETS
# --------------------------------------------------
train_p1 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_1")
train_p2 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_2")
train_p3 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_3")
train_p4 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_4")

f_name1 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_important_features2")

train_all = (train_p1
             .merge(train_p2)
             .merge(train_p3)
             .merge(train_p4))


In [4]:
import geemap

# Radio Features
# remove_list = ['6_VV', '3_VH', '5_VV', '6_DPSVIm', '4_VH', 'VH_p50', 'DPSVIm_p5',
#                '5_DPSVIm', 'VH_p5', '4_DPSVIm', '5_VH', 'RVI_p5', 'VV/VH_p50',
#                'NDPI_p50', '6_VV-VH', '4_VV', '5_VV-VH', '7_VH', '6_VH', '2_VH']


year        = '2022'
date_start  = ee.Date(f'{year}-01-01')
date_end    = ee.Date(f'{year}-12-31')
cloud_cover = 50
cloud_prob  = 75
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']


feat_select = ee.List(ee.FeatureCollection(f_name1).aggregate_array('0')).distinct()
# feat_select = feat_select.removeAll(ee.List(remove_list))

model = ee.Classifier.smileRandomForest(
    numberOfTrees=150,
    variablesPerSplit=None,
    minLeafPopulation=2,
    bagFraction=0.5,
    maxNodes=None,
    seed=0
)



In [5]:
model = model.train(
    features = ee.FeatureCollection(train_all).filter(ee.Filter.eq('Lbl_cls_major', 1)),
    classProperty = 'Label_clas',
    inputProperties = feat_select
)

## Inference

### Inference

In [6]:
regions = {
    "spain": ee.Geometry.Rectangle([-5.1, 41.9, -4.9, 42.1]),
    "france": ee.Geometry.Rectangle([1.95, 48.05, 2.05, 48.20]),
    "austria": ee.Geometry.Rectangle([16.70, 48.10, 16.80, 48.20]),
    "romania": ee.Geometry.Rectangle([27.25, 44.20, 27.35, 44.28]),
    #"almaty": ee.Geometry.Rectangle([])
}

roi = regions["austria"].buffer(100)


cloudprob = (ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
            .filterBounds(roi)
            .filterDate(date_start, date_end))

dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(roi)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

# Join by system:index; keep first cloud prob img as property "Cloud"
idJoin    = ee.Filter.equals(leftField='system:index', rightField='system:index')
innerJoin = ee.Join.saveFirst('Cloud')
dataSE = ee.ImageCollection(innerJoin.apply(primary=dataSE, secondary=cloudprob, condition=idJoin))

# Preprocess and immediately CLIP to ROI
dataSE = (preproS2(dataSE, cloud_prob)
          .select(bands)
          .map(lambda img: ee.Image(img).clip(roi)))

# Spectral indices (on already-clipped images)
s2Idx = vegetation_indices(dataSE)

# Monthly S2  toBands
months = ee.List.sequence(3, 10)
S2_Monthly = generateS2month(months, s2Idx, year).toBands()

S2_Yearly = ee.Image(s2Idx.reduce(ee.Reducer.percentile([5, 50, 98])))
S2_Yearly = S2_Yearly.rename(newNames(S2_Yearly, year)).clip(roi)

# Sentinel-1 (ROI-first)
radar = (ee.ImageCollection("COPERNICUS/S1_GRD")
        .filterBounds(roi)
        .filterDate('2022-01-01', '2023-01-01')
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV'))
        .select(['VV','VH']))

radar = preproS1(radar, roi)
radar = s1_features(radar)

# Monthly S1 + Yearly S1
S1_Monthly = generateS1month(radar, '2022-01-01', 12, 1, 'month').toBands()
S1_Yearly  = radar.reduce(ee.Reducer.percentile([5, 50, 98])).clip(roi)

# LST + DEM (ROI-first)
lst = (ee.ImageCollection('MODIS/061/MOD21C3')
      .filterBounds(roi)
      .filterDate('2022-01-01', '2022-12-31')
      .select('LST_Day'))
lst = clipic(lst, roi).toBands()

dem = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(roi)

In [7]:
def test_feat_select_on_image(img, feat_select):
    feat_list = ee.List(feat_select)
    bands     = ee.List(img.bandNames())

    missing = feat_list.removeAll(bands)
    extra   = bands.removeAll(feat_list)

    print("========== TEST 1: BAND CHECK ==========")
    print("Expected (feat_select):", feat_list.getInfo())
    print("Image bands           :", bands.getInfo())
    print("Missing bands         :", missing.getInfo())
    print("Extra bands           :", extra.getInfo())

    if missing.size().getInfo() == 0:
        print("======================================")
        print("     1st Test Is Successful")
        print("======================================")
        return True
    else:
        raise ValueError(
            f"ERROR (Test 1): Missing bands in image: {missing.getInfo()}"
        )


def pixel_stats(img, scale=10):

    # 1 = all bands valid, 0 = missing at least one band
    completeness = img.mask().reduce(ee.Reducer.min()).rename('comp')

    # FULL PIXELS: where completeness == 1
    full = completeness.eq(1).selfMask().reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=img.geometry(),
        scale=scale,
        maxPixels=1e9,
        bestEffort=True,
        tileScale=4
    ).get('comp')

    # TOTAL PIXELS: all pixels (completeness unmasked)
    total = completeness.unmask().reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=img.geometry(),
        scale=scale,
        maxPixels=1e9,
        bestEffort=True,
        tileScale=4
    ).get('comp')

    full    = ee.Number(full)
    total   = ee.Number(total)
    missing = total.subtract(full)

    return {
        'full': full,
        'missing': missing,
        'total': total
    }


def test_missing_pixels(img, scale=10, threshold=0.1, label="IMAGE"):

    stats = pixel_stats(img, scale=scale)

    full    = ee.Number(stats['full'])
    missing = ee.Number(stats['missing'])
    total   = ee.Number(stats['total'])

    total_val = total.getInfo()
    if total_val == 0:
        raise ValueError(f"{label}: total pixel count is 0, image mask/ROI is empty.")

    missing_ratio = missing.divide(total)

    print(f"========== TEST 2: MISSING PIXELS ({label}) ==========")
    print("Full pixels     :", full.getInfo())
    print("Missing pixels  :", missing.getInfo())
    print("Total pixels    :", total_val)
    print("Missing ratio   :", missing_ratio.getInfo())

    if missing_ratio.lte(threshold).getInfo():
        print("======================================")
        print("     2nd Test Is Successful")
        print("======================================")
        return
    else:
        raise ValueError(
            f"{label}: too many missing pixels: "
            f"{missing_ratio.getInfo()*100:.2f}% "
            f"(allowed ≤ {threshold*100}%)"
        )


data_stack = ee.Image.cat([
    S2_Monthly,
    S2_Yearly,
    S1_Monthly,
    S1_Yearly,
    lst,
    dem
]).clip(roi)


test_feat_select_on_image(data_stack, feat_select)
final_data = data_stack.select(feat_select)

feature_mask = final_data.mask().reduce(ee.Reducer.min())


class_map  = final_data.classify(model).rename('class')
class_map_remap = rename_classes(class_map)

class_map_for_test = class_map_remap.updateMask(feature_mask)

styled_rgb = Visualization(class_map_for_test)

test_missing_pixels(
    img=final_data,
    scale=10,              # Sentinel-2 native resolution
    threshold=0.20,
    label="INPUT IMAGE (features)"
)

test_missing_pixels(
    img=class_map_for_test,
    scale=10,
    threshold=0.20,
    label="OUTPUT IMAGE (classification, masked like features)"
)


========== TEST 1: BAND CHECK ==========
Expected (feat_select): ['0_2022_B3', '0_2022_B4', '0_2022_B5', '0_2022_BLFEI', '0_2022_LAI', '0_2022_LAIg', '0_2022_LCCI', '0_2022_MSAVI', '0_2022_NDVI', '0_2022_SAVI', '0_2022_SRNIRR', '0_2022_gndvi', '1_2022_B6', '1_2022_B8', '1_2022_BLFEI', '1_2022_DIRESWIR', '1_2022_EVI2', '1_2022_LAI', '1_2022_LAIg', '1_2022_LCCI', '1_2022_MNDWI', '1_2022_MSAVI', '1_2022_NDTI', '1_2022_NDVI', '1_2022_SAVI', '1_2022_SRNIRR', '1_2022_gndvi', '2022_01_01_LST_Day', '2022_02_01_LST_Day', '2022_03_01_LST_Day', '2022_04_01_LST_Day', '2022_05_01_LST_Day', '2022_06_01_LST_Day', '2022_07_01_LST_Day', '2022_08_01_LST_Day', '2022_09_01_LST_Day', '2022_10_01_LST_Day', '2022_11_01_LST_Day', '2022_12_01_LST_Day', '2022_B11_p98', '2022_B12_p50', '2022_B12_p98', '2022_B2_p50', '2022_B3_p50', '2022_B4_p50', '2022_B5_p50', '2022_B6_p98', '2022_B7_p98', '2022_BLFEI_p5', '2022_BLFEI_p50', '2022_BSI_p5', '2022_BSI_p50', '2022_BSI_p98', '2022_DIRESWIR_p50', '2022_EVI2_p5', '2022

     2nd Test Is Successful


### Visualization

In [8]:
m = geemap.Map(center=[48, 67], zoom=4)
truecolor = (dataSE.median()
             .select(['B4','B3','B2'])
             .visualize(min=0, max=3000))
m.add_layer(truecolor, {}, 'S2 True Color (median)', opacity=0.6)

m.add_layer(styled_rgb, {}, 'EU CropMap 2022 (classified)', opacity=1.0)

# Build legend from SLD
items = legend_items_from_sld()
legend_dict = {it['label']: it['color'] for it in items}
m.add_legend(title='EU Crop Classes', legend_dict=legend_dict, position='bottomright')
m.centerObject(roi, 12)
m

Map(center=[48.149994462167164, 16.749999999998302], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
type(class_map_for_test)

ee.image.Image

In [10]:
true_labels_raw = (
    ee.ImageCollection('JRC/D5/EUCROPMAP/V1')
      .filterBounds(roi)
      .filterDate('2022-01-01', '2023-01-01')
      .first()
)

roi_geom = roi
true_labels_raw = true_labels_raw.clip(roi_geom)
true_labels = true_labels_raw.select(0).rename('true')

minor_classes = [
    211,212,213,214,215,216,217,218,219,
    221,222,223,
    230,231,232,233,
    240,250,290
]

minor_mask = true_labels.remap(
    minor_classes,
    ee.List.repeat(1, len(minor_classes)),
    0
).eq(1)

true_minor = true_labels.updateMask(minor_mask)


In [11]:
pred = class_map_for_test.clip(roi_geom).rename('pred')
pred_minor = pred.updateMask(minor_mask)

In [12]:
# 1 where pred == true, 0 otherwise
match = pred_minor.eq(true_minor).rename('match')

stats = match.reduceRegion(
    reducer=ee.Reducer.sum().combine(ee.Reducer.count(), '', True),
    geometry=roi_geom,
    scale=10,
    maxPixels=1e10,
    tileScale=4
)

matches = ee.Number(stats.get('match_sum')).getInfo()   # pixels where pred == true
total   = ee.Number(stats.get('match_count')).getInfo() # total evaluated pixels
accuracy = int(matches)/total


In [13]:
print('Matches:', int(matches))
print('Total  :', total)
print('Accuracy within ROI:', accuracy)

if accuracy >= 0.65:
  print("======================================")
  print("     3rd Test Is Successful")
  print("======================================")
else:
  print("======================================")
  print("     3rd Test Is not Successful")
  print("======================================")

Matches: 448312
Total  : 521729
Accuracy within ROI: 0.8592813510462328
     3rd Test Is Successful
